In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

path = r"C:/Users/paulo/Documents/Pro/ENSAE/Statapp/Datasets/histo_gazpar_merged.csv"
df = pd.read_csv(path)


In [10]:
print(f"Données chargées : {df.shape[0]:,} lignes × {df.shape[1]} colonnes")
print(f"Période : {df['date'].min()} → {df['date'].max()}")
print(f"Nombre de PCE : {df['pce'].nunique():,}")

Données chargées : 1,358,668 lignes × 35 colonnes
Période : 2024-07-01T00:00:00.000Z → 2025-05-31T00:00:00.000Z
Nombre de PCE : 7,090


In [11]:
# %% EXPLORATION INITIALE
print("="*70)
print("APERÇU DES DONNÉES")
print("="*70)

# Affichage des premières lignes
print("\nPremières lignes :")
display(df.head(10))

# Types de données
print("\nTypes de colonnes :")
print(df.dtypes)

# Valeurs manquantes
print("\nValeurs manquantes :")
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({
    'Colonnes': missing.index,
    'Valeurs manquantes': missing.values,
    'Pourcentage': missing_pct.values
})
print(missing_df[missing_df['Valeurs manquantes'] > 0])

# Statistiques descriptives
print("\nStatistiques descriptives :")
print(df[['valeur_energie_conso', 'temperature', 'hdd']].describe())

# Vérification des valeurs aberrantes connues
print("\nVérification des pics du 1er et 28 avril 2025 :")
anomalies = df[df['date'].isin(['2025-04-01', '2025-04-28'])]
if len(anomalies) > 0:
    top_conso = anomalies.nlargest(3, 'valeur_energie_conso')[['date', 'pce', 'valeur_energie_conso']]
    print(top_conso)
else:
    print("Dates non trouvées dans le dataset")

APERÇU DES DONNÉES

Premières lignes :


,code_station_meteo_pitd,pce,pce_releve,pitd,freq_rel,segmnt,profil,car,business_u_finale,freq_rel_1,...,code_station_meteo_pitd_char,date,temperature,hdd,mois,jour,temperature_de_reference,hdd_ref,row_number,ntile100
0,59343001,1105643886865,1105643886865,GD0991,1M,RES,P011,3801,DGP,1M,...,59343001,2024-07-01T00:00:00.000Z,17.118833,0.000000,7,1,20.24,0.00,1396,1
1,59343001,1113892899627,1113892899627,GD0991,1M,RES,P012,8787,DGP,1M,...,59343001,2024-11-07T00:00:00.000Z,8.472167,8.027833,11,7,9.96,6.54,3437,1
2,59343001,1104052034976,1104052034976,GD0991,1M,RES,P012,10185,DGP,1M,...,59343001,2024-07-01T00:00:00.000Z,17.118833,0.000000,7,1,20.24,0.00,1010,1
3,59343001,1100578870486,1100578870486,GD0991,1M,RES,P012,8633,DGP,1M,...,59343001,2024-12-18T00:00:00.000Z,10.585000,5.915000,12,18,5.50,11.00,136,1
4,59343001,1119247397883,1119247397883,GD0991,1M,RES,P012,7922,DGP,1M,...,59343001,2024-07-03T00:00:00.000Z,15.730500,0.769500,7,3,20.41,0.00,4759,1
5,59343001,1124167802873,1124167802873,GD0991,1M,RES,P012,5779,DGP,1M,...,59343001,2025-05-13T00:00:00.000Z,18.103000,0.000000,5,13,14.94,1.56,6019,1
6,59343001,1114182199810,1114182199810,GD0991,1M,RES,P011,2000,DGP,1M,...,59343001,2024-07-03T00:00:00.000Z,15.730500,0.769500,7,3,20.41,0.00,3485,1
7,59343001,1110564337978,1110564337978,GD0991,1M,RES,P011,3824,DGP,1M,...,59343001,2024-10-08T00:00:00.000Z,15.218500,1.281500,10,8,14.28,2.22,2611,1
8,59343001,1119392154915,1119392154915,GD0991,1M,RES,P012,11698,DGP,1M,...,59343001,2024-07-03T00:00:00.000Z,15.730500,0.769500,7,3,20.41,0.00,4808,1
9,59343001,1110853728588,1110853728588,GD0991,1M,RES,P012,7447,DGP,1M,...,59343001,2024-10-08T00:00:00.000Z,15.218500,1.281500,10,8,14.28,2.22,2663,1



Types de colonnes :
code_station_meteo_pitd           int64
pce                               int64
pce_releve                        int64
pitd                             object
freq_rel                         object
segmnt                           object
profil                           object
car                               int64
business_u_finale                object
freq_rel_1                       object
valeur_energie_conso            float64
date_parsed                      object
conso_theorique                 float64
delta_conso                     float64
business_u_finale_new            object
year                              int64
month                             int64
day                               int64
gasday                           object
date_insert                      object
date_debut_periode_parsed       float64
date_fin_periode_parsed         float64
horodate_debut_conso_local      float64
horodate_fin_conso_local        float64
event              

In [16]:
df_work = df.copy()

# Conversion des dates (suppression du timezone UTC)
df_work["gasday"] = pd.to_datetime(df_work["gasday"], utc=True).dt.tz_convert(None)
df_work["date"]   = pd.to_datetime(df_work["date"], utc=True).dt.tz_convert(None)

In [19]:
print(f"Dates converties")
print(f"   - Type gasday : {df_work['gasday'].dtype}")
print(f"   - Type date   : {df_work['date'].dtype}")
print(f"   - Période     : {df_work['date'].min()} → {df_work['date'].max()}")

# %% EXPLORATION INITIALE
print("\n" + "="*70)
print("APERÇU DES DONNÉES")
print("="*70)

# Affichage des premières lignes
print("\nPremières lignes :")
print(df_work.head(10))

# Types de données
print("\nTypes de colonnes :")
print(df_work.dtypes)

# Valeurs manquantes
print("\nValeurs manquantes :")
missing = df_work.isnull().sum()
missing_pct = (missing / len(df_work) * 100).round(2)
missing_df = pd.DataFrame({
    'Colonnes': missing.index,
    'Valeurs manquantes': missing.values,
    'Pourcentage (%)': missing_pct.values
})
print(missing_df[missing_df['Valeurs manquantes'] > 0])

# Statistiques descriptives sur les variables clés
print("\nStatistiques descriptives :")
stats_cols = ['valeur_energie_conso', 'temperature', 'hdd', 'hdd_ref']
print(df_work[stats_cols].describe().round(2))

# Distribution des valeurs nulles dans la consommation
print(f"\nValeurs de consommation = 0 : {(df_work['valeur_energie_conso'] == 0).sum():,} ({(df_work['valeur_energie_conso'] == 0).mean()*100:.1f}%)")

# Vérification des pics du 1er et 28 avril 2025
print("\nVérification des anomalies connues (1er et 28 avril 2025) :")
anomaly_dates = ['2025-04-01', '2025-04-28']
for date_str in anomaly_dates:
    subset = df_work[df_work['date'] == date_str]
    if len(subset) > 0:
        max_conso = subset['valeur_energie_conso'].max()
        max_pce = subset.loc[subset['valeur_energie_conso'].idxmax(), 'pce']
        print(f"   {date_str} : max = {max_conso:,.0f} kWh (PCE: {max_pce})")
    else:
        print(f"   {date_str} : Date non trouvée")

Dates converties
   - Type gasday : datetime64[ns]
   - Type date   : datetime64[ns]
   - Période     : 2024-07-01 00:00:00 → 2025-05-31 00:00:00

APERÇU DES DONNÉES

Premières lignes :
   code_station_meteo_pitd            pce     pce_releve    pitd freq_rel  \
0                 59343001  1105643886865  1105643886865  GD0991       1M   
1                 59343001  1113892899627  1113892899627  GD0991       1M   
2                 59343001  1104052034976  1104052034976  GD0991       1M   
3                 59343001  1100578870486  1100578870486  GD0991       1M   
4                 59343001  1119247397883  1119247397883  GD0991       1M   
5                 59343001  1124167802873  1124167802873  GD0991       1M   
6                 59343001  1114182199810  1114182199810  GD0991       1M   
7                 59343001  1110564337978  1110564337978  GD0991       1M   
8                 59343001  1119392154915  1119392154915  GD0991       1M   
9                 59343001  1110853728588  1

In [61]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.mixed_linear_model import MixedLM

In [63]:
# Date d'origine : 1er juillet 2024
date_origine = pd.to_datetime('2024-07-01')

# Calcul du nombre de jours depuis l'origine
df_work['trend'] = (df_work['date'] - date_origine).dt.days

# Variables nécessaires pour le modèle baseline
required_vars = ['pce', 'date', 'valeur_energie_conso', 'hdd', 'trend']

In [69]:
for var in required_vars:
    if var in df_work.columns:
        nb_missing = df_work[var].isnull().sum()
        print(f"   ✓ {var:25s} : {nb_missing:>10,} valeurs manquantes ({nb_missing/len(df_work)*100:.2f}%)")
    else:
        print(f"   ✗ {var:25s} : MANQUANTE !")

# Statistiques sur les variables clés
print(f"\nStatistiques descriptives :")
print(df_work[['valeur_energie_conso', 'hdd', 'trend']].describe().round(2))

   ✓ pce                       :          0 valeurs manquantes (0.00%)
   ✓ date                      :          0 valeurs manquantes (0.00%)
   ✓ valeur_energie_conso      :          0 valeurs manquantes (0.00%)
   ✓ hdd                       :          0 valeurs manquantes (0.00%)
   ✓ trend                     :          0 valeurs manquantes (0.00%)

Statistiques descriptives :
       valeur_energie_conso         hdd       trend
count            1358668.00  1358668.00  1358668.00
mean                  23.94        5.06      159.00
std                 1150.06        5.22      101.69
min                    0.00        0.00        0.00
25%                    2.00        0.00       69.00
50%                    7.00        3.76      150.00
75%                   32.00        8.48      238.00
max               947789.00       17.53      334.00


In [70]:
# Sélection des colonnes nécessaires (version minimale)
cols_model = ['pce', 'date', 'valeur_energie_conso', 'hdd', 'trend']

df_model = df_work[cols_model].copy()

### Baseline Lineae Mixed Effect Model

In [72]:
# Conversion du PCE en string (pour les effets aléatoires)
df_model['pce'] = df_model['pce'].astype(str)

print("""
Modèle : valeur_energie_conso ~ hdd + trend

Effets fixes (FE) :
  - β₀ : Intercept global
  - β₁ : Sensibilité moyenne au froid (HDD)
  - β₂ : Tendance temporelle

Effets aléatoires (RE) par PCE :
  - Random intercept : b₀ⱼ (niveau de conso propre à chaque PCE)
  - Random slope HDD : b₁ⱼ (sensibilité au froid spécifique)

Structure : 
  valeur_energie_conso_ij = (β₀ + b₀ⱼ) + (β₁ + b₁ⱼ)·hdd + β₂·trend + εᵢⱼ
""")


Modèle : valeur_energie_conso ~ hdd + trend

Effets fixes (FE) :
  - β₀ : Intercept global
  - β₁ : Sensibilité moyenne au froid (HDD)
  - β₂ : Tendance temporelle

Effets aléatoires (RE) par PCE :
  - Random intercept : b₀ⱼ (niveau de conso propre à chaque PCE)
  - Random slope HDD : b₁ⱼ (sensibilité au froid spécifique)

Structure : 
  valeur_energie_conso_ij = (β₀ + b₀ⱼ) + (β₁ + b₁ⱼ)·hdd + β₂·trend + εᵢⱼ



In [73]:
import time
start_time = time.time()

# Spécification du modèle
model = MixedLM.from_formula(
    formula='valeur_energie_conso ~ hdd + trend',
    groups='pce',
    re_formula='~hdd',  # Random intercept + random slope sur HDD
    data=df_model
)

# Estimation via REML
result = model.fit(method='lbfgs', reml=True)

elapsed_time = time.time() - start_time

C:\Users\paulo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\paulo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\regression\mixed_linear_model.py:2206: ConvergenceWarning: MixedLM optimization failed, trying a different optimizer may help.
  warnings.warn(msg, ConvergenceWarning)
C:\Users\paulo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\regression\mixed_linear_model.py:2218: ConvergenceWarning: Gradient optimization failed, |grad| = 8690.247090
  warnings.warn(msg, ConvergenceWarning)


In [74]:
# Résumé complet
print(result.summary())

              Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: valeur_energie_conso
No. Observations: 1358668 Method:             REML                
No. Groups:       7090    Scale:              1321725.1037        
Min. group size:  1       Log-Likelihood:     -11502914.7020      
Max. group size:  254     Converged:          No                  
Mean group size:  191.6                                           
-------------------------------------------------------------------
                    Coef.   Std.Err.    z     P>|z|  [0.025  0.975]
-------------------------------------------------------------------
Intercept            1.511     1.930   0.783  0.434  -2.272   5.293
hdd                  4.186     0.200  20.934  0.000   3.794   4.578
trend                0.008     0.010   0.741  0.459  -0.013   0.028
pce Var            642.447     0.192                               
pce x hdd Cov      -29.411     0.017                               
hd

Note : L'algorithme ne converge pas au vu de la valeur du gradient. Impossible d'avoir des coefficients stables en présence d'aussi gros outliers. On va les traiter : 
- IQR
- Isolation Forest

### I. IQR

In [ ]:
sub = df_work['valeur_energie_conso']

Q1 = df_work['valeur_energie_conso'].quantile(0.25)
Q3 = df_work['valeur_energie_conso'].quantile(0.75)

In [78]:
Q1

np.float64(2.0)

In [ ]:
q = df_work.groupby('date')['valeur_energie_conso'].quantile([0.25,0.50, 0.75]).unstack()
q.columns =  ['Q1', 'Q2_median', 'Q3']
q['IQR'] = q['Q3'] - q['Q1']
k = 3
q['lower_bound'] = q['Q1'] - k * q['IQR']
q['upper_bound'] = q['Q3'] + k * q['IQR']


In [105]:
q.head(10)

,Q1,Q2_median,Q3,IQR,lower_bound,upper_bound
date,,,,,,
2024-07-01,0.0,3.0,6.0,6.0,-18.0,24.0
2024-07-02,1.0,3.0,6.0,5.0,-14.0,21.0
2024-07-03,1.0,3.0,6.0,5.0,-14.0,21.0
2024-07-04,1.0,3.0,6.0,5.0,-14.0,21.0
2024-07-05,0.0,3.0,6.0,6.0,-18.0,24.0
2024-07-06,0.0,3.0,6.0,6.0,-18.0,24.0
2024-07-07,0.0,3.0,7.0,7.0,-21.0,28.0
2024-07-08,0.0,3.0,6.0,6.0,-18.0,24.0
2024-07-09,0.0,3.0,6.0,6.0,-18.0,24.0


Note : Le problème qui va apparaître avec IQR, c'estb qu on va commencer à supprimer des PCE qui sont simplement de gros consommateurs... Il faudrait prendre également en compte le comportement global et moyen de chaque PCE, c'est à ce moment que Isolation Forest devient peut etre utile !

In [114]:
df_model.head(10)

,pce,date,valeur_energie_conso,hdd,trend
0,1105643886865,2024-07-01,0.0,0.000000,0
1,1113892899627,2024-11-07,29.0,8.027833,129
2,1104052034976,2024-07-01,2.0,0.000000,0
3,1100578870486,2024-12-18,19.0,5.915000,170
4,1119247397883,2024-07-03,19.0,0.769500,2
5,1124167802873,2025-05-13,3.0,0.000000,316
6,1114182199810,2024-07-03,21.0,0.769500,2
7,1110564337978,2024-10-08,0.0,1.281500,99
8,1119392154915,2024-07-03,0.0,0.769500,2
9,1110853728588,2024-10-08,1.0,1.281500,99


In [115]:
df_model_with_bounds = df_model.merge(q[['lower_bound', 'upper_bound']], left_on = 'date', right_index=True, how = 'left')

In [116]:
df_model_with_bounds.head(10)

,pce,date,valeur_energie_conso,hdd,trend,lower_bound,upper_bound
0,1105643886865,2024-07-01,0.0,0.000000,0,-18.0,24.0
1,1113892899627,2024-11-07,29.0,8.027833,129,-87.0,144.0
2,1104052034976,2024-07-01,2.0,0.000000,0,-18.0,24.0
3,1100578870486,2024-12-18,19.0,5.915000,170,-78.0,146.0
4,1119247397883,2024-07-03,19.0,0.769500,2,-14.0,21.0
5,1124167802873,2025-05-13,3.0,0.000000,316,-14.0,21.0
6,1114182199810,2024-07-03,21.0,0.769500,2,-14.0,21.0
7,1110564337978,2024-10-08,0.0,1.281500,99,-22.0,34.0
8,1119392154915,2024-07-03,0.0,0.769500,2,-14.0,21.0
9,1110853728588,2024-10-08,1.0,1.281500,99,-22.0,34.0


In [124]:
df_correct = df_model_with_bounds[(df_model_with_bounds['valeur_energie_conso'] >= df_model_with_bounds['lower_bound']) & (df_model_with_bounds['valeur_energie_conso'] <= df_model_with_bounds['upper_bound'])]
df_deleted = df_model_with_bounds[~((df_model_with_bounds['valeur_energie_conso'] >= df_model_with_bounds['lower_bound']) & (df_model_with_bounds['valeur_energie_conso'] <= df_model_with_bounds['upper_bound']))]

In [125]:
df_clean = df_correct[['pce', 'date', 'valeur_energie_conso', 'hdd', 'trend']]
df_clean

,pce,date,valeur_energie_conso,hdd,trend
0,1105643886865,2024-07-01,0.0,0.000000,0
1,1113892899627,2024-11-07,29.0,8.027833,129
2,1104052034976,2024-07-01,2.0,0.000000,0
3,1100578870486,2024-12-18,19.0,5.915000,170
4,1119247397883,2024-07-03,19.0,0.769500,2
...,...,...,...,...,...
1358663,1107959318880,2024-09-03,4.0,0.000000,64
1358664,1112445595150,2024-09-03,10.0,0.000000,64
1358665,1114905840557,2024-09-03,0.0,0.000000,64
1358666,1114182241365,2024-07-17,3.0,0.000000,16


In [126]:
df_deleted

,pce,date,valeur_energie_conso,hdd,trend,lower_bound,upper_bound
35,1127351542976,2025-05-14,36.0,0.000000,317,-17.0,25.0
67,1106222746867,2024-10-02,92.0,3.407000,93,-40.0,58.0
245,1103762643668,2025-05-16,30.0,3.141500,319,-17.0,25.0
368,1126772716291,2024-08-13,18.0,0.000000,43,-12.0,16.0
419,1128364600534,2024-09-10,23.0,1.893167,71,-14.0,21.0
...,...,...,...,...,...,...,...
1357904,1101736591459,2025-05-19,33.0,1.601833,322,-20.0,29.0
1357953,1115774224678,2024-09-28,65.0,6.205833,89,-34.0,50.0
1358242,1111866839000,2024-09-17,26.0,0.327500,78,-17.0,25.0
1358373,1105643863045,2024-07-06,25.0,0.550000,5,-18.0,24.0


In [127]:
start_time = time.time()

# Spécification du modèle
model2 = MixedLM.from_formula(
    formula='valeur_energie_conso ~ hdd + trend',
    groups='pce',
    re_formula='~hdd',  # Random intercept + random slope sur HDD
    data=df_clean
)

# Estimation via REML
result2 = model.fit(method='lbfgs', reml=True)

elapsed_time = time.time() - start_time

C:\Users\paulo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\paulo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\regression\mixed_linear_model.py:2206: ConvergenceWarning: MixedLM optimization failed, trying a different optimizer may help.
  warnings.warn(msg, ConvergenceWarning)
C:\Users\paulo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\regression\mixed_linear_model.py:2218: ConvergenceWarning: Gradient optimization failed, |grad| = 8690.247090
  warnings.warn(msg, ConvergenceWarning)


In [128]:
df_model

,pce,date,valeur_energie_conso,hdd,trend
0,1105643886865,2024-07-01,0.0,0.000000,0
1,1113892899627,2024-11-07,29.0,8.027833,129
2,1104052034976,2024-07-01,2.0,0.000000,0
3,1100578870486,2024-12-18,19.0,5.915000,170
4,1119247397883,2024-07-03,19.0,0.769500,2
...,...,...,...,...,...
1358663,1107959318880,2024-09-03,4.0,0.000000,64
1358664,1112445595150,2024-09-03,10.0,0.000000,64
1358665,1114905840557,2024-09-03,0.0,0.000000,64
1358666,1114182241365,2024-07-17,3.0,0.000000,16


In [129]:
# %% VÉRIFICATION : LES OUTLIERS DE 947K ONT-ILS ÉTÉ SUPPRIMÉS ?
print("="*70)
print("VÉRIFICATION DU NETTOYAGE")
print("="*70)

# Statistiques AVANT nettoyage
print(f"\n📊 Dataset AVANT nettoyage (df_model) :")
print(f"   Nombre de lignes : {len(df_model):,}")
print(f"   Max consommation : {df_model['valeur_energie_conso'].max():,.0f} kWh")
print(f"   Statistiques :")
print(df_model['valeur_energie_conso'].describe())

# Statistiques APRÈS nettoyage
print(f"\n📊 Dataset APRÈS nettoyage (df_clean) :")
print(f"   Nombre de lignes : {len(df_clean):,}")
print(f"   Max consommation : {df_clean['valeur_energie_conso'].max():,.0f} kWh")
print(f"   Statistiques :")
print(df_clean['valeur_energie_conso'].describe())

# Différence
print(f"\n✅ Lignes supprimées : {len(df_model) - len(df_clean):,}")

# Vérification spécifique des valeurs > 10,000 kWh
print(f"\n🔎 Valeurs > 10,000 kWh :")
high_values_before = (df_model['valeur_energie_conso'] > 10000).sum()
high_values_after = (df_clean['valeur_energie_conso'] > 10000).sum()
print(f"   Avant nettoyage : {high_values_before} lignes")
print(f"   Après nettoyage : {high_values_after} lignes")

# Vérification spécifique des valeurs > 1,000 kWh
print(f"\n🔎 Valeurs > 1,000 kWh :")
high_values_before_1k = (df_model['valeur_energie_conso'] > 1000).sum()
high_values_after_1k = (df_clean['valeur_energie_conso'] > 1000).sum()
print(f"   Avant nettoyage : {high_values_before_1k} lignes")
print(f"   Après nettoyage : {high_values_after_1k} lignes")

# Top 10 des valeurs les plus élevées APRÈS nettoyage
print(f"\n🔝 Top 10 des valeurs APRÈS nettoyage (df_clean) :")
print(df_clean.nlargest(10, 'valeur_energie_conso')[['date', 'pce', 'valeur_energie_conso', 'hdd', 'trend']])

# Recherche spécifique des PCE problématiques connus
pce_problematique_1 = '1114761118314'
pce_problematique_2 = '1124023097737'

print(f"\n🔎 Recherche des PCE problématiques connus :")
print(f"   PCE {pce_problematique_1} dans df_clean : {pce_problematique_1 in df_clean['pce'].values}")
print(f"   PCE {pce_problematique_2} dans df_clean : {pce_problematique_2 in df_clean['pce'].values}")

if pce_problematique_1 in df_clean['pce'].values:
    print(f"\n   ⚠️ Le PCE {pce_problematique_1} est encore présent !")
    subset = df_clean[df_clean['pce'] == pce_problematique_1]
    print(f"   Nombre de lignes : {len(subset)}")
    print(subset[['date', 'valeur_energie_conso', 'hdd']].sort_values('valeur_energie_conso', ascending=False).head())

if pce_problematique_2 in df_clean['pce'].values:
    print(f"\n   ⚠️ Le PCE {pce_problematique_2} est encore présent !")
    subset = df_clean[df_clean['pce'] == pce_problematique_2]
    print(f"   Nombre de lignes : {len(subset)}")
    print(subset[['date', 'valeur_energie_conso', 'hdd']].sort_values('valeur_energie_conso', ascending=False).head())

VÉRIFICATION DU NETTOYAGE

📊 Dataset AVANT nettoyage (df_model) :
   Nombre de lignes : 1,358,668
   Max consommation : 947,789 kWh
   Statistiques :
count    1.358668e+06
mean     2.394162e+01
std      1.150060e+03
min      0.000000e+00
25%      2.000000e+00
50%      7.000000e+00
75%      3.200000e+01
max      9.477890e+05
Name: valeur_energie_conso, dtype: float64

📊 Dataset APRÈS nettoyage (df_clean) :
   Nombre de lignes : 1,346,091
   Max consommation : 270 kWh
   Statistiques :
count    1.346091e+06
mean     2.182886e+01
std      3.128903e+01
min      0.000000e+00
25%      2.000000e+00
50%      7.000000e+00
75%      3.100000e+01
max      2.700000e+02
Name: valeur_energie_conso, dtype: float64

✅ Lignes supprimées : 12,577

🔎 Valeurs > 10,000 kWh :
   Avant nettoyage : 2 lignes
   Après nettoyage : 0 lignes

🔎 Valeurs > 1,000 kWh :
   Avant nettoyage : 3 lignes
   Après nettoyage : 0 lignes

🔝 Top 10 des valeurs APRÈS nettoyage (df_clean) :
              date            pce  valeu

In [130]:
# %% FILTRAGE DES PCE AVEC PEU D'OBSERVATIONS
print("="*70)
print("FILTRAGE DES PCE (minimum d'observations requis)")
print("="*70)

# Compter le nombre d'observations par PCE
nb_obs_par_pce = df_clean.groupby('pce').size()

print(f"\n📊 Distribution du nombre d'observations par PCE :")
print(nb_obs_par_pce.describe())

# Histogramme visuel
print(f"\n📊 Répartition :")
bins = [0, 10, 30, 50, 100, 200, 300]
hist = pd.cut(nb_obs_par_pce, bins=bins).value_counts().sort_index()
print(hist)

# Seuil minimum
seuil_min = 30  # Au moins 30 jours de données

pce_to_keep = nb_obs_par_pce[nb_obs_par_pce >= seuil_min].index
nb_pce_before = df_clean['pce'].nunique()

df_clean_filtered = df_clean[df_clean['pce'].isin(pce_to_keep)].copy()

nb_pce_after = df_clean_filtered['pce'].nunique()
nb_pce_removed = nb_pce_before - nb_pce_after
nb_lignes_removed = len(df_clean) - len(df_clean_filtered)

print(f"\n✅ Filtrage avec seuil = {seuil_min} observations/PCE :")
print(f"   PCE avant         : {nb_pce_before:,}")
print(f"   PCE après         : {nb_pce_after:,}")
print(f"   PCE retirés       : {nb_pce_removed:,} ({nb_pce_removed/nb_pce_before*100:.1f}%)")
print(f"   Lignes avant      : {len(df_clean):,}")
print(f"   Lignes après      : {len(df_clean_filtered):,}")
print(f"   Lignes retirées   : {nb_lignes_removed:,} ({nb_lignes_removed/len(df_clean)*100:.1f}%)")

print(f"\n📊 Nouvelle distribution des observations par PCE :")
nb_obs_final = df_clean_filtered.groupby('pce').size()
print(nb_obs_final.describe())

FILTRAGE DES PCE (minimum d'observations requis)

📊 Distribution du nombre d'observations par PCE :
count    7086.000000
mean      189.964860
std        82.539729
min         1.000000
25%       118.000000
50%       248.000000
75%       251.000000
max       254.000000
dtype: float64

📊 Répartition :
(0, 10]        108
(10, 30]       199
(30, 50]       543
(50, 100]      704
(100, 200]    1030
(200, 300]    4502
Name: count, dtype: int64

✅ Filtrage avec seuil = 30 observations/PCE :
   PCE avant         : 7,086
   PCE après         : 6,790
   PCE retirés       : 296 (4.2%)
   Lignes avant      : 1,346,091
   Lignes après      : 1,341,553
   Lignes retirées   : 4,538 (0.3%)

📊 Nouvelle distribution des observations par PCE :
count    6790.000000
mean      197.577761
std        75.622767
min        30.000000
25%       133.000000
50%       249.000000
75%       251.000000
max       254.000000
dtype: float64


In [131]:
# %% STANDARDISATION DES VARIABLES
print("="*70)
print("STANDARDISATION DES VARIABLES")
print("="*70)

# Statistiques avant standardisation
print(f"\n📊 Statistiques AVANT standardisation :")
print(df_clean_filtered[['valeur_energie_conso', 'hdd', 'trend']].describe().round(2))

# Standardisation (z-score)
df_clean_filtered['hdd_scaled'] = (
    (df_clean_filtered['hdd'] - df_clean_filtered['hdd'].mean()) / 
    df_clean_filtered['hdd'].std()
)

df_clean_filtered['trend_scaled'] = (
    (df_clean_filtered['trend'] - df_clean_filtered['trend'].mean()) / 
    df_clean_filtered['trend'].std()
)

print(f"\n📊 Statistiques APRÈS standardisation :")
print(df_clean_filtered[['hdd_scaled', 'trend_scaled']].describe().round(2))

print(f"\n✅ Variables standardisées créées : hdd_scaled, trend_scaled")
print(f"   (mean ≈ 0, std ≈ 1 pour chacune)")

STANDARDISATION DES VARIABLES

📊 Statistiques AVANT standardisation :
       valeur_energie_conso         hdd       trend
count            1341553.00  1341553.00  1341553.00
mean                  21.86        5.08      158.58
std                   31.30        5.23      101.44
min                    0.00        0.00        0.00
25%                    2.00        0.00       69.00
50%                    7.00        3.76      150.00
75%                   32.00        8.68      238.00
max                  270.00       17.53      334.00

📊 Statistiques APRÈS standardisation :
       hdd_scaled  trend_scaled
count  1341553.00    1341553.00
mean        -0.00          0.00
std          1.00          1.00
min         -0.97         -1.56
25%         -0.97         -0.88
50%         -0.25         -0.08
75%          0.69          0.78
max          2.38          1.73

✅ Variables standardisées créées : hdd_scaled, trend_scaled
   (mean ≈ 0, std ≈ 1 pour chacune)


In [132]:
# %% MODÈLE LMM SIMPLIFIÉ - RANDOM INTERCEPT SEULEMENT
print("="*70)
print("MODÈLE LMM SIMPLIFIÉ (Random Intercept seulement)")
print("="*70)

import time
from statsmodels.regression.mixed_linear_model import MixedLM

# Préparation des données
df_for_lmm = df_clean_filtered[['pce', 'date', 'valeur_energie_conso', 
                                  'hdd_scaled', 'trend_scaled']].copy()
df_for_lmm['pce'] = df_for_lmm['pce'].astype(str)

print(f"\n✅ Dataset prêt pour LMM :")
print(f"   Observations : {len(df_for_lmm):,}")
print(f"   PCE          : {df_for_lmm['pce'].nunique():,}")
print(f"   Max conso    : {df_for_lmm['valeur_energie_conso'].max():.2f} kWh")

print(f"\n📝 Modèle : valeur_energie_conso ~ hdd_scaled + trend_scaled")
print(f"   Effets aléatoires : Random Intercept seulement (pas de random slope)")

print(f"\n⏳ Estimation en cours (devrait être plus rapide)...")
start = time.time()

# Modèle SIMPLIFIÉ : Random intercept seulement
model_simple = MixedLM.from_formula(
    formula='valeur_energie_conso ~ hdd_scaled + trend_scaled',
    groups='pce',
    re_formula='~1',  # ← SEULEMENT random intercept (pas de slope)
    data=df_for_lmm
)

# Estimation
result_simple = model_simple.fit(method='lbfgs', reml=True, maxiter=1000)

elapsed = time.time() - start

print(f"\n✅ Estimation terminée en {elapsed/60:.1f} minutes")

# Affichage des résultats
print("\n" + "="*70)
print("RÉSULTATS DU MODÈLE SIMPLIFIÉ")
print("="*70)
print(result_simple.summary())

# Extraction des coefficients
print("\n" + "="*70)
print("COEFFICIENTS FIXES")
print("="*70)
fe_table = pd.DataFrame({
    'Coefficient': result_simple.fe_params,
    'Std.Err': result_simple.bse_fe,
    'z-value': result_simple.tvalues,
    'P>|z|': result_simple.pvalues
})
print(fe_table.round(6))

# Interprétation
print("\n📌 Interprétation (avec variables standardisées) :")
beta_hdd = result_simple.fe_params['hdd_scaled']
beta_trend = result_simple.fe_params['trend_scaled']

# Dé-standardisation pour interprétation
std_hdd = df_clean_filtered['hdd'].std()
std_trend = df_clean_filtered['trend'].std()

beta_hdd_original = beta_hdd / std_hdd
beta_trend_original = beta_trend / std_trend

print(f"   β (hdd_scaled)   = {beta_hdd:.4f} → +1 écart-type de HDD = +{beta_hdd:.2f} kWh")
print(f"   β (trend_scaled) = {beta_trend:.4f} → +1 écart-type de trend = {beta_trend:.2f} kWh")
print(f"\n   En échelle originale :")
print(f"   β (hdd)   ≈ {beta_hdd_original:.4f} kWh par degré-jour")
print(f"   β (trend) ≈ {beta_trend_original:.6f} kWh par jour")
print(f"                → {beta_trend_original*365:.2f} kWh par an")

print(f"\n   Variance résiduelle (scale) : {result_simple.scale:.2f}")
print(f"   (Rappel : Irina avait ≈ 4,700)")

MODÈLE LMM SIMPLIFIÉ (Random Intercept seulement)

✅ Dataset prêt pour LMM :
   Observations : 1,341,553
   PCE          : 6,790
   Max conso    : 270.00 kWh

📝 Modèle : valeur_energie_conso ~ hdd_scaled + trend_scaled
   Effets aléatoires : Random Intercept seulement (pas de random slope)

⏳ Estimation en cours (devrait être plus rapide)...


C:\Users\paulo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
C:\Users\paulo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\regression\mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)



✅ Estimation terminée en 0.1 minutes

RÉSULTATS DU MODÈLE SIMPLIFIÉ
               Mixed Linear Model Regression Results
Model:              MixedLM Dependent Variable: valeur_energie_conso
No. Observations:   1341553 Method:             REML                
No. Groups:         6790    Scale:              313.5479            
Min. group size:    30      Log-Likelihood:     inf                 
Max. group size:    254     Converged:          Yes                 
Mean group size:    197.6                                           
--------------------------------------------------------------------
             Coef.   Std.Err.     z     P>|z|    [0.025     0.975]  
--------------------------------------------------------------------
Intercept    -0.000 499635.381   -0.000 1.000 -979267.353 979267.353
hdd_scaled   22.428      0.016 1362.183 0.000      22.396     22.460
trend_scaled -1.323      0.017  -76.000 0.000      -1.357     -1.289
pce Var       0.000                               

In [133]:
# %% LMM AVEC VARIABLES ORIGINALES (NON STANDARDISÉES)
print("="*70)
print("LMM SIMPLIFIÉ - VARIABLES ORIGINALES")
print("="*70)

import time
from statsmodels.regression.mixed_linear_model import MixedLM

# Préparation des données (variables ORIGINALES)
df_for_lmm_v2 = df_clean_filtered[['pce', 'date', 'valeur_energie_conso', 
                                     'hdd', 'trend']].copy()
df_for_lmm_v2['pce'] = df_for_lmm_v2['pce'].astype(str)

# Supprimer les NaN
df_for_lmm_v2 = df_for_lmm_v2.dropna()

print(f"\n✅ Dataset prêt :")
print(f"   Observations : {len(df_for_lmm_v2):,}")
print(f"   PCE          : {df_for_lmm_v2['pce'].nunique():,}")

print(f"\n📝 Modèle : valeur_energie_conso ~ hdd + trend")
print(f"   Effets aléatoires : Random Intercept seulement")

print(f"\n⏳ Estimation en cours...")
start = time.time()

# Modèle avec variables ORIGINALES
model_v2 = MixedLM.from_formula(
    formula='valeur_energie_conso ~ hdd + trend',
    groups='pce',
    re_formula='~1',  # Random intercept seulement
    data=df_for_lmm_v2
)

# Estimation
result_v2 = model_v2.fit(method='lbfgs', reml=True, maxiter=1000)

elapsed = time.time() - start
print(f"\n✅ Estimation terminée en {elapsed/60:.1f} minutes")

# Résultats
print("\n" + "="*70)
print("RÉSULTATS - VARIABLES ORIGINALES")
print("="*70)
print(result_v2.summary())

# Coefficients
print("\n" + "="*70)
print("COEFFICIENTS FIXES")
print("="*70)
fe_table_v2 = pd.DataFrame({
    'Coefficient': result_v2.fe_params,
    'Std.Err': result_v2.bse_fe,
    'z-value': result_v2.tvalues,
    'P>|z|': result_v2.pvalues,
    'CI 95% inf': result_v2.conf_int()[0],
    'CI 95% sup': result_v2.conf_int()[1]
})
print(fe_table_v2.round(6))

# Interprétation
print("\n📌 INTERPRÉTATION :")
beta_hdd = result_v2.fe_params['hdd']
beta_trend = result_v2.fe_params['trend']
beta_intercept = result_v2.fe_params['Intercept']

print(f"\n   β₀ (Intercept) = {beta_intercept:.4f} kWh")
print(f"   β₁ (hdd)       = {beta_hdd:.4f} kWh par degré-jour")
print(f"                    → 1 degré-jour de plus = +{beta_hdd:.2f} kWh/jour")
print(f"\n   β₂ (trend)     = {beta_trend:.6f} kWh par jour")
print(f"                    → Évolution quotidienne = {beta_trend:.4f} kWh/jour")
print(f"                    → Évolution annuelle    = {beta_trend*365:.2f} kWh/an/client")

if beta_trend < 0:
    print(f"\n   🎯 DESTRUCTION DE DEMANDE DÉTECTÉE :")
    print(f"      Baisse de {abs(beta_trend*365):.2f} kWh/an par client")
    print(f"      Soit {abs(beta_trend*365)/np.mean(df_for_lmm_v2['valeur_energie_conso'])*100:.2f}% de la consommation moyenne")

# Variances
print("\n" + "="*70)
print("VARIANCES")
print("="*70)
print(f"   Variance intercept (σ²_u0) : {result_v2.cov_re.iloc[0, 0]:.2f}")
print(f"   Variance résiduelle (σ²_ε) : {result_v2.scale:.2f}")

# Comparaison avec Irina
print(f"\n📊 Comparaison avec Irina (slide 8) :")
print(f"   Irina  : β(hdd) ≈ 4.69, scale ≈ 4,708")
print(f"   Toi    : β(hdd) = {beta_hdd:.2f}, scale = {result_v2.scale:.2f}")

LMM SIMPLIFIÉ - VARIABLES ORIGINALES

✅ Dataset prêt :
   Observations : 1,341,553
   PCE          : 6,790

📝 Modèle : valeur_energie_conso ~ hdd + trend
   Effets aléatoires : Random Intercept seulement

⏳ Estimation en cours...


C:\Users\paulo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
C:\Users\paulo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\regression\mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)



✅ Estimation terminée en 0.1 minutes

RÉSULTATS - VARIABLES ORIGINALES
              Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: valeur_energie_conso
No. Observations: 1341553 Method:             REML                
No. Groups:       6790    Scale:              313.5479            
Min. group size:  30      Log-Likelihood:     inf                 
Max. group size:  254     Converged:          Yes                 
Mean group size:  197.6                                           
------------------------------------------------------------------
           Coef.   Std.Err.     z     P>|z|    [0.025     0.975]  
------------------------------------------------------------------
Intercept   0.000 499635.381    0.000 1.000 -979267.353 979267.353
hdd         4.288      0.003 1362.183 0.000       4.282      4.294
trend      -0.013      0.000  -76.000 0.000      -0.013     -0.013
pce Var     0.000                                                 


COE

In [134]:
# %% LMM COMPLET - RANDOM INTERCEPT + RANDOM SLOPE
print("="*70)
print("LMM COMPLET - RANDOM INTERCEPT + RANDOM SLOPE SUR HDD")
print("="*70)

import time
from statsmodels.regression.mixed_linear_model import MixedLM

print(f"\n📝 Modèle : valeur_energie_conso ~ hdd + trend")
print(f"   Effets fixes : β₀ (intercept), β₁ (hdd), β₂ (trend)")
print(f"   Effets aléatoires : b₀ⱼ (intercept par PCE) + b₁ⱼ (slope HDD par PCE)")
print(f"\n   Structure complète :")
print(f"   y_ij = (β₀ + b₀ⱼ) + (β₁ + b₁ⱼ)·hdd + β₂·trend + ε_ij")

print(f"\n⏳ Estimation en cours (peut prendre 5-10 minutes)...")
start = time.time()

# Modèle COMPLET
model_full = MixedLM.from_formula(
    formula='valeur_energie_conso ~ hdd + trend',
    groups='pce',
    re_formula='~hdd',  # ← Random intercept + random slope
    data=df_for_lmm_v2
)

# Estimation avec plus d'itérations si besoin
try:
    result_full = model_full.fit(method='lbfgs', reml=True, maxiter=2000)
    converged = True
except Exception as e:
    print(f"\n❌ Erreur lors de l'estimation : {e}")
    converged = False

if converged:
    elapsed = time.time() - start
    print(f"\n✅ Estimation terminée en {elapsed/60:.1f} minutes")
    
    # Résultats
    print("\n" + "="*70)
    print("RÉSULTATS - MODÈLE COMPLET")
    print("="*70)
    print(result_full.summary())
    
    # Coefficients fixes
    print("\n" + "="*70)
    print("COEFFICIENTS FIXES")
    print("="*70)
    fe_table_full = pd.DataFrame({
        'Coefficient': result_full.fe_params,
        'Std.Err': result_full.bse_fe,
        'z-value': result_full.tvalues,
        'P>|z|': result_full.pvalues,
        'CI 95% inf': result_full.conf_int()[0],
        'CI 95% sup': result_full.conf_int()[1]
    })
    print(fe_table_full.round(6))
    
    # Interprétation
    print("\n📌 INTERPRÉTATION DES COEFFICIENTS FIXES :")
    beta_hdd = result_full.fe_params['hdd']
    beta_trend = result_full.fe_params['trend']
    beta_intercept = result_full.fe_params['Intercept']
    
    print(f"\n   β₀ (Intercept) = {beta_intercept:.4f} kWh")
    print(f"   β₁ (hdd)       = {beta_hdd:.4f} kWh par degré-jour (effet moyen population)")
    print(f"   β₂ (trend)     = {beta_trend:.6f} kWh par jour")
    print(f"                    → Destruction de demande = {beta_trend*365:.2f} kWh/an")
    
    # Variances des effets aléatoires
    print("\n" + "="*70)
    print("VARIANCES DES EFFETS ALÉATOIRES")
    print("="*70)
    
    var_intercept = result_full.cov_re.iloc[0, 0]
    var_slope = result_full.cov_re.iloc[1, 1]
    cov_intercept_slope = result_full.cov_re.iloc[0, 1]
    var_residuelle = result_full.scale
    
    print(f"   Variance intercept (σ²_u0)     : {var_intercept:.2f}")
    print(f"   Variance slope HDD (σ²_u1)     : {var_slope:.2f}")
    print(f"   Covariance intercept-slope     : {cov_intercept_slope:.2f}")
    
    # Corrélation
    if var_intercept > 0 and var_slope > 0:
        corr = cov_intercept_slope / np.sqrt(var_intercept * var_slope)
        print(f"   Corrélation intercept-slope    : {corr:.4f}")
    
    print(f"   Variance résiduelle (σ²_ε)     : {var_residuelle:.2f}")
    
    # Comparaison avec Irina
    print("\n" + "="*70)
    print("COMPARAISON AVEC IRINA (slide 8)")
    print("="*70)
    print(f"   {'Métrique':<30} {'Irina':<15} {'Toi':<15}")
    print(f"   {'-'*60}")
    print(f"   {'β (hdd)':<30} {'4.691':<15} {beta_hdd:<15.3f}")
    print(f"   {'β (trend)':<30} {'-0.004':<15} {beta_trend:<15.6f}")
    print(f"   {'Variance intercept':<30} {'1221.092':<15} {var_intercept:<15.2f}")
    print(f"   {'Variance slope HDD':<30} {'202.618':<15} {var_slope:<15.2f}")
    print(f"   {'Covariance':<30} {'-481.739':<15} {cov_intercept_slope:<15.2f}")
    print(f"   {'Scale (variance résiduelle)':<30} {'4708.926':<15} {var_residuelle:<15.2f}")
    
    # Interprétation de l'hétérogénéité
    if var_slope > 0:
        print("\n📊 INTERPRÉTATION DE L'HÉTÉROGÉNÉITÉ :")
        std_slope = np.sqrt(var_slope)
        print(f"   Écart-type du slope HDD : {std_slope:.2f} kWh/degré-jour")
        print(f"   → 95% des PCE ont une sensibilité entre {beta_hdd - 1.96*std_slope:.2f} et {beta_hdd + 1.96*std_slope:.2f} kWh/degré-jour")
        print(f"   → Cela confirme une forte hétérogénéité entre PCE (logements différents)")
else:
    print("\n❌ Le modèle n'a pas convergé")

LMM COMPLET - RANDOM INTERCEPT + RANDOM SLOPE SUR HDD

📝 Modèle : valeur_energie_conso ~ hdd + trend
   Effets fixes : β₀ (intercept), β₁ (hdd), β₂ (trend)
   Effets aléatoires : b₀ⱼ (intercept par PCE) + b₁ⱼ (slope HDD par PCE)

   Structure complète :
   y_ij = (β₀ + b₀ⱼ) + (β₁ + b₁ⱼ)·hdd + β₂·trend + ε_ij

⏳ Estimation en cours (peut prendre 5-10 minutes)...

✅ Estimation terminée en 0.3 minutes

RÉSULTATS - MODÈLE COMPLET
              Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: valeur_energie_conso
No. Observations: 1341553 Method:             REML                
No. Groups:       6790    Scale:              132.8627            
Min. group size:  30      Log-Likelihood:     -5212225.8057       
Max. group size:  254     Converged:          Yes                 
Mean group size:  197.6                                           
-------------------------------------------------------------------
                  Coef.   Std.Err.     z      P>

In [135]:
# %% SAUVEGARDE DU DATASET NETTOYÉ
print("="*70)
print("SAUVEGARDE DU DATASET NETTOYÉ")
print("="*70)

# Sauvegarder le dataset nettoyé (pour réutilisation)
output_path = r"C:/Users/paulo/Documents/Pro/ENSAE/Statapp/Datasets/df_clean_v1.csv"
df_clean_filtered.to_csv(output_path, index=False)

print(f"✅ Dataset sauvegardé : {output_path}")
print(f"   Lignes : {len(df_clean_filtered):,}")
print(f"   PCE    : {df_clean_filtered['pce'].nunique():,}")

SAUVEGARDE DU DATASET NETTOYÉ
✅ Dataset sauvegardé : C:/Users/paulo/Documents/Pro/ENSAE/Statapp/Datasets/df_clean_v1.csv
   Lignes : 1,341,553
   PCE    : 6,790


In [141]:
# %% SAUVEGARDE DES RÉSULTATS DU MODÈLE BASELINE (VERSION CORRIGÉE)
print("="*70)
print("SAUVEGARDE DES RÉSULTATS - MODÈLE BASELINE")
print("="*70)

# Calculer le nombre de groupes à partir du dataset
n_groups = df_for_lmm_v2['pce'].nunique()

# Créer un dictionnaire avec tous les résultats
results_baseline = {
    'model_name': 'LMM_complet_baseline',
    'formula': 'valeur_energie_conso ~ hdd + trend',
    're_formula': '~hdd',
    'n_obs': int(result_full.nobs),
    'n_groups': int(n_groups),  # ← Calculé manuellement
    'converged': True,
    
    # Coefficients fixes
    'beta_intercept': float(result_full.fe_params['Intercept']),
    'beta_hdd': float(result_full.fe_params['hdd']),
    'beta_trend': float(result_full.fe_params['trend']),
    
    # Erreurs standard
    'se_intercept': float(result_full.bse_fe['Intercept']),
    'se_hdd': float(result_full.bse_fe['hdd']),
    'se_trend': float(result_full.bse_fe['trend']),
    
    # Variances
    'var_intercept': float(result_full.cov_re.iloc[0, 0]),
    'var_slope_hdd': float(result_full.cov_re.iloc[1, 1]),
    'cov_intercept_slope': float(result_full.cov_re.iloc[0, 1]),
    'var_residual': float(result_full.scale),
    
    # Métriques
    'log_likelihood': float(result_full.llf),
    'aic': float(result_full.aic),
    'bic': float(result_full.bic),
    
    # Destruction de demande
    'destruction_demande_kwh_jour': float(result_full.fe_params['trend']),
    'destruction_demande_kwh_an': float(result_full.fe_params['trend'] * 365)
}

# Sauvegarder en JSON
import json
output_json = r"C:/Users/paulo/Documents/Pro/ENSAE/Statapp/Results/baseline_results.json"

# Créer le dossier si nécessaire
import os
os.makedirs(os.path.dirname(output_json), exist_ok=True)

with open(output_json, 'w', encoding='utf-8') as f:
    json.dump(results_baseline, f, indent=4, ensure_ascii=False)

print(f"✅ Résultats sauvegardés : {output_json}")

# Afficher un résumé
print("\n" + "="*70)
print("RÉSUMÉ DU MODÈLE BASELINE")
print("="*70)
print(f"\n📊 Données :")
print(f"   Observations : {results_baseline['n_obs']:,}")
print(f"   PCE (groupes) : {results_baseline['n_groups']:,}")

print(f"\n📈 Coefficients fixes :")
print(f"   β₀ (Intercept) : {results_baseline['beta_intercept']:.4f} kWh")
print(f"   β₁ (hdd)       : {results_baseline['beta_hdd']:.4f} kWh/degré-jour")
print(f"   β₂ (trend)     : {results_baseline['beta_trend']:.6f} kWh/jour")

print(f"\n🎯 Destruction de demande :")
print(f"   Par jour : {results_baseline['destruction_demande_kwh_jour']:.4f} kWh")
print(f"   Par an   : {results_baseline['destruction_demande_kwh_an']:.2f} kWh/client")

pct_destruction = (abs(results_baseline['destruction_demande_kwh_an']) / 
                   df_for_lmm_v2['valeur_energie_conso'].mean()) * 100
print(f"   Soit     : {pct_destruction:.2f}% de la consommation moyenne")

print(f"\n📐 Variances :")
print(f"   σ²_u0 (intercept)      : {results_baseline['var_intercept']:.2f}")
print(f"   σ²_u1 (slope HDD)      : {results_baseline['var_slope_hdd']:.2f}")
print(f"   Covariance             : {results_baseline['cov_intercept_slope']:.2f}")
print(f"   σ²_ε (résiduelle)      : {results_baseline['var_residual']:.2f}")

print(f"\n📊 Métriques :")
print(f"   Log-Likelihood : {results_baseline['log_likelihood']:.2f}")
print(f"   AIC            : {results_baseline['aic']:.2f}")
print(f"   BIC            : {results_baseline['bic']:.2f}")

SAUVEGARDE DES RÉSULTATS - MODÈLE BASELINE
✅ Résultats sauvegardés : C:/Users/paulo/Documents/Pro/ENSAE/Statapp/Results/baseline_results.json

RÉSUMÉ DU MODÈLE BASELINE

📊 Données :
   Observations : 1,341,553
   PCE (groupes) : 6,790

📈 Coefficients fixes :
   β₀ (Intercept) : 2.6760 kWh
   β₁ (hdd)       : 4.0653 kWh/degré-jour
   β₂ (trend)     : -0.013344 kWh/jour

🎯 Destruction de demande :
   Par jour : -0.0133 kWh
   Par an   : -4.87 kWh/client
   Soit     : 22.28% de la consommation moyenne

📐 Variances :
   σ²_u0 (intercept)      : 17.97
   σ²_u1 (slope HDD)      : 7.53
   Covariance             : -3.36
   σ²_ε (résiduelle)      : 132.86

📊 Métriques :
   Log-Likelihood : -5212225.81
   AIC            : nan
   BIC            : nan
